In [ ]:
# Only use one thread
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

# Do not use GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
import pandas as pd
import numpy as np

import inspect
import random
import pickle
import math
import textwrap
import time
import warnings

from scipy.stats import pearsonr, mode
from sklearn.base import clone
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression, Lasso
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.metrics import mean_squared_error, roc_auc_score, f1_score, accuracy_score, mean_absolute_error, log_loss
from sklearn.model_selection import cross_val_score, RepeatedKFold, KFold
from sklearn.svm import SVR
from tqdm import tqdm

import xgboost as xgb

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor

import tensorflow as tf

import scipy.stats

from utils import *

## Get Data

In [ ]:
USE_INDIV_SURVEY_VARS = True
USE_IMPORTANCE_ITEMS = False

df = pd.read_csv('../research-data/processed/lak22-courseload-final-studydata.csv')

ADDITIONAL_INDIV_VARS = [
    'course_name_number', 'is_stem_course', 'is_stem_student', 'course_student_stem_match',
     'n_satisfied_prereqs_2021_Spring', 'n_satisfied_prereqs_all_past_semesters',
    'percent_satisfied_prereqs_2021_Spring', 'percent_satisfied_prereqs_all_past_semesters',
    'is_non_letter_grade_course', 'student_gpa', 'student_gpa_major', 
    'tl_importance', 'me_importance', 'ps_importance', 'combined_importance', 
    'tl_manage', 'me_manage', 'ps_manage', 'cl_combined_manage'
]
if not USE_IMPORTANCE_ITEMS:
    for var in ['tl_importance', 'me_importance', 'ps_importance', 'combined_importance']:
        del df[var]

if not USE_INDIV_SURVEY_VARS:
    for var in ADDITIONAL_INDIV_VARS:
        del df[var]

# Remove string section information
for col in ['section_num','secondary_section_number','all_section_numbers']:
    if col in df.columns:
        del df[col]
        
# Remove Labels that are not needed
for col in ['tl2', 'tl_sensitivity', 'me_sensitivity', 'ps_sensitivity', 'cl_sensitivity',
            'tl1_smoothed_lmm', 'me_smoothed_lmm', 'ps_smoothed_lmm', 'cl_smoothed_lmm', 
            'tl1_smoothed_student_average', 'me_smoothed_student_average', 'ps_smoothed_student_average',
            'cl_smoothed_student_average']:
    if col in df.columns:
        del df[col]

# Drop string columns and get dummies for string var
df = df.set_index('course_name_number')
df = pd.get_dummies(df, columns=['class_type']) # upper, lower division, grad

# Train (CV) and holdout
train, test = train_test_split(df, test_size=0.15, random_state=12345, shuffle=True)

## CV Error Control Variables

In [ ]:
with open('./models/model-results-25-control variables.p', 'rb') as f:
    prelim = pickle.load(f)

In [ ]:
pd.set_option('display.max_rows', 500)
out = pd.concat([get_sorted_model_result_table(prelim, l) for l in LABELS])
display(out)
pd.set_option('display.max_rows', 25)

## CV Error KNN

In [ ]:
with open('./models/model-results-25-knn.p', 'rb') as f:
    prelim = pickle.load(f)

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', 500)
out = pd.concat([get_sorted_model_result_table(prelim, l) for l in LABELS])
display(out)
pd.set_option('display.max_rows', 25)

## Test set error

In [ ]:
with open('./models/model-results-25-control variables.p', 'rb') as f:
    prelim = pickle.load(f)

ignore_warnings=True
if ignore_warnings:
        warnings.filterwarnings("ignore")

res = dict()
for l in LABELS: res[l] = dict()    
    
for target in tqdm(['tl1', 'me', 'ps', 'cl_combined']):
    for model in ['random', 'linreg', 'rf', 'xgb', 'enet', 'svm', 'nn']:
        res[target][model] = apply_model(prelim, train.copy(), test.copy(), 
                                         target=target, model_ref=model,
                                        imputing_strategy='control variables')

# Add ensemble
for target in tqdm(['tl1', 'me', 'ps', 'cl_combined']):
    temp = []
    for model in ['linreg', 'rf', 'xgb', 'enet', 'svm', 'nn']:
        temp.append(res[target][model][0])
    res[target]['ensemble'] = (list(map(np.mean, zip(*temp))), res[target]['linreg'][1])

In [ ]:
# How is test set error related to credit hour vs. predicted load discrepancy?
def correlation_pairwise_complete(series1, series2, print_n=False):
    x, y = series1.values, series2.values
    nas = np.logical_or(np.isnan(x), np.isnan(y))
    corr = scipy.stats.pearsonr(x[~nas], y[~nas])
    if print_n:
        print(f'N = {len(x[~nas])}')
    return corr

def standardize_z(v):
    return (v - np.mean(v)) / np.std(v)
    
tmp = pd.DataFrame({
    'pred': res['cl_combined']['ensemble'][0],
    'label': res['cl_combined']['ensemble'][1],
    'n_credit_hours': test['n_credit_hours']
})
tmp['mae'] = np.abs(tmp['pred'] - tmp['label'])
tmp['pred_z'] = standardize_z(tmp['pred'])
tmp['n_credit_hours_z'] = standardize_z(tmp['n_credit_hours'])
tmp['pred_cred_discrepancy_z'] = tmp['pred_z'] - tmp['n_credit_hours_z']
display(tmp.head(3))
correlation_pairwise_complete(tmp['mae'], tmp['pred_cred_discrepancy_z'])

In [ ]:
plt.plot(tmp['mae'], tmp['pred_cred_discrepancy_z'], 'o')

In [ ]:
def bootstrap_score(gold, preds, fun=mean_squared_error, n_iter=1000, alpha=0.05, reference=0.8):
    n_obs = len(gold)
    true = fun(gold, preds)
    out = []
    for _ in range(n_iter):
        mask = np.random.randint(n_obs, size=n_obs)
        try:
            tmp = fun(np.array(gold)[mask], preds[mask])
        except:
            continue
        out.append(tmp)
    p = 1 - (sum([mae < reference for mae in out])/len(out))
    return true, np.quantile(out, alpha/2), np.quantile(out, 1-(alpha/2)), p

In [ ]:
# Evaluate 
targets, models, maes, boot_maes = [], [], [], []
    
for target in tqdm(['tl1', 'me', 'ps', 'cl_combined']):
    baseline = mean_absolute_error(res[target]['random'][0], res[target]['random'][1])
    for model in ['random', 'linreg', 'rf', 'xgb', 'enet', 'svm', 'nn', 'ensemble']:
        targets.append(target); models.append(model)
        maes.append(mean_absolute_error(res[target][model][0], res[target][model][1]))
        boot_maes.append(bootstrap_score(res[target][model][0], res[target][model][1], 
                                         fun=mean_absolute_error, n_iter=10000,
                                         reference=baseline))
    

In [ ]:
metrics = pd.DataFrame({'target': targets, 'model': models, 'mae': maes,
                        'lower': [b[1] for b in boot_maes],
                        'upper': [b[2] for b in boot_maes],
                        'p': [b[3] for b in boot_maes]})
metrics.sort_values(by=['target', 'mae'], inplace=True)

In [ ]:
# Add error reduction, percentage error, and model -> rank
res2 = []
for target in ['tl1', 'me', 'ps', 'cl_combined']:
    tmp = metrics[metrics['target'] == target].copy()
    tmp = tmp.sort_values(by='mae')
    baseline = tmp[tmp['model'] == 'random'].mae.values[0]
    tmp['mae_diff'] = baseline - tmp['mae']
    tmp['mae_percent_improve_to_random'] = tmp['mae_diff'] * 100 / baseline
    tmp['model_rank'] = list(range(1, tmp.shape[0]+1))
    res2.append(tmp)
ans = pd.concat(res2)

In [ ]:
round(ans, 3)

In [ ]:
# Average model rank across constructs
ans.groupby('model')['model_rank'].mean().sort_values().to_frame()

# Correlation betwen MAE and discrepancy on full-coded data for robustness

In [ ]:
with open('./models/model-results-25-control variables.p', 'rb') as f:
    prelim = pickle.load(f)

full = pd.concat([train, test])
    
ignore_warnings=True
if ignore_warnings:
        warnings.filterwarnings("ignore")

res = dict()
for l in LABELS: res[l] = dict()    
    
for target in tqdm(['tl1', 'me', 'ps', 'cl_combined']):
    for model in ['random', 'linreg', 'rf', 'xgb', 'enet', 'svm', 'nn']:
        res[target][model] = apply_model(prelim, train.copy(), full.copy(), 
                                         target=target, model_ref=model,
                                        imputing_strategy='control variables')

# Add ensemble
for target in tqdm(['tl1', 'me', 'ps', 'cl_combined']):
    temp = []
    for model in ['linreg', 'rf', 'xgb', 'enet', 'svm', 'nn']:
        temp.append(res[target][model][0])
    res[target]['ensemble'] = (list(map(np.mean, zip(*temp))), res[target]['linreg'][1])

In [ ]:
tmp = pd.DataFrame({
    'pred': res['cl_combined']['ensemble'][0],
    'label': res['cl_combined']['ensemble'][1],
    'n_credit_hours': full['n_credit_hours']
})
tmp['mae'] = np.abs(tmp['pred'] - tmp['label'])
tmp['pred_z'] = standardize_z(tmp['pred'])
tmp['n_credit_hours_z'] = standardize_z(tmp['n_credit_hours'])
tmp['pred_cred_discrepancy_z'] = tmp['pred_z'] - tmp['n_credit_hours_z']

print(tmp.shape)

correlation_pairwise_complete(tmp['mae'], tmp['pred_cred_discrepancy_z'])